In [31]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [32]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

### 1st Retrival tool 

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [26]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)

In [27]:
wiki.name

'wikipedia'

### 2nd Retrival tool

In [15]:
## Web Based Loader

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
embeddings = OllamaEmbeddings(model="llama3")

In [8]:
loader = WebBaseLoader("https://the-good-doctor.fandom.com/wiki/Shaun_Murphy")
docs = loader.load()


In [9]:
docs

[Document(page_content='\n\n\n\nShaun Murphy | The Good Doctor Wiki | Fandom\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Good Doctor Wiki\n\n\n\n\n\n Explore\n\n \n\n\n\n\n Main Page\n\n\n\n\n Discuss\n\n\n\n\nAll Pages\n\n\n\n\nCommunity\n\n\n\n\nInteractive Maps\n\n\n\n\nRecent Blog Posts\n\n\n\n\n\n\n\n\nCharacters\n\n \n\n\n\n\nMain Characters\n \n\n\n\n\nShaun Murphy\n\n\n\n\nLea Dilallo\n\n\n\n\nMorgan Reznick\n\n\n\n\nAlex Park\n\n\n\n\nAudrey Lim\n\n\n\n\nAsher Wolke\n\n\n\n\nJordan Allen\n\n\n\n\nMarcus Andrews\n\n\n\n\nAaron Glassman\n\n\n\n\nJared Kalu\n\n\n\n\n\n\n\nRetired Characters\n \n\n\n\n\nClaire Browne\n\n\n\n\nCarly Lever\n\n\n\n\nOlivia Jackson\n\n\n\n\nMateo Rendón Osma\n\n\n\n\nSalen Morrison\n\n\n\n\n\n\n\n\n\n\n\nEpisodes\n\n \n\n\n\n\nSeason 1\n \n\n\n\n\n1.01 Burnt Food\n\n\n\n\n1.02 Mount Rushmore\n\n\n\n\n1.03 Oliver\n\n\n\n\n1.04 Pipes\n\n\n\n\n1.05 Point Three Percent\n\n\n\n\n1.06 Not Fake\n\n\n

In [12]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)

In [13]:
vectordb

In [14]:
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000166742E13C0>)

In [19]:
from langchain.tools.retriever import create_retriever_tool
retreval_tool=create_retriever_tool(retriever,"Shaun Murphy",
                                    "Search details for Shaun Murphy. For any question related to Shaun Murphy use this tool. ")


In [20]:
retreval_tool.name

'Shaun Murphy'

### 3rd Retreval tool

In [21]:
# Arxiv tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [23]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv= ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [25]:
arxiv.name

'arxiv'

In [29]:
tools=[retreval_tool,wiki,arxiv]

### Agents

In [58]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [59]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [60]:
from langchain.agents import create_tool_calling_agent,create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

### Agent Executor

In [61]:
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [64]:
agent_executor.invoke({"input":"Tell me about Shaun Murphy","tools": tools})



> Entering new AgentExecutor chain...


ValueError: Ollama call failed with status code 400. Details: {"error":"invalid options: tools"}